# Test Redshift Database

In [ ]:
import os
import glob
import numpy as np
from astropy.io import fits
from astropy.table import Table
from desiutil.log import get_logger, DEBUG, INFO
from desispec.io.meta import faflavor2program
import desispec.database.redshift as dsr

In [ ]:
specprod = os.environ['SPECPROD'] = 'f5'

## Consistency checks on exposures, frames and tiles

In [ ]:
tiles = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.csv'), format='ascii.csv')

In [ ]:
tiles

In [ ]:
# with fits.open(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits')) as hdulist:
#     exposures = hdulist['EXPOSURES'].data
#     frames = hdulist['FRAMES'].data
exposures = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='EXPOSURES')
frames = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='FRAMES')

In [ ]:
exposures

In [ ]:
for k, expid in enumerate(exposures['EXPID']):
    assert (frames['NIGHT'][frames['EXPID'] == expid] == exposures[k]['NIGHT']).all()
    assert (frames['TILEID'][frames['EXPID'] == expid] == exposures[k]['TILEID']).all()
    assert (frames['TILERA'][frames['EXPID'] == expid] == exposures[k]['TILERA']).all()
    assert (frames['TILEDEC'][frames['EXPID'] == expid] == exposures[k]['TILEDEC']).all()
    # assert (frames['MJD'][frames['EXPID'] == expid] == exposures[k]['MJD']).all()
    # assert (frames['EXPTIME'][frames['EXPID'] == expid] == exposures[k]['EXPTIME']).all()
    assert (frames['AIRMASS'][frames['EXPID'] == expid] == exposures[k]['AIRMASS']).all()
    # assert (frames['EBV'][frames['EXPID'] == expid] == exposures[k]['EBV']).all()
    assert (frames['SEEING_ETC'][frames['EXPID'] == expid] == exposures[k]['SEEING_ETC']).all()
    assert (frames['EFFTIME_ETC'][frames['EXPID'] == expid] == exposures[k]['EFFTIME_ETC']).all()
    # assert (frames['TSNR2_GPBDARK'][frames['EXPID'] == expid] == exposures[k]['TSNR2_GPBDARK']).all()
    # assert (frames['TSNR2_ELG'][frames['EXPID'] == expid] == exposures[k]['TSNR2_ELG']).all()
    # assert (frames['TSNR2_GPBBRIGHT'][frames['EXPID'] == expid] == exposures[k]['TSNR2_GPBBRIGHT']).all()
    # assert (frames['TSNR2_LYA'][frames['EXPID'] == expid] == exposures[k]['TSNR2_LYA']).all()
    # assert (frames['TSNR2_BGS'][frames['EXPID'] == expid] == exposures[k]['TSNR2_BGS']).all()
    # assert (frames['TSNR2_GPBBACKUP'][frames['EXPID'] == expid] == exposures[k]['TSNR2_GPBBACKUP']).all()
    # assert (frames['TSNR2_QSO'][frames['EXPID'] == expid] == exposures[k]['TSNR2_QSO']).all()
    # assert (frames['TSNR2_LRG'][frames['EXPID'] == expid] == exposures[k]['TSNR2_LRG']).all()
    assert (frames['SURVEY'][frames['EXPID'] == expid] == exposures[k]['SURVEY']).all()
    assert (frames['GOALTYPE'][frames['EXPID'] == expid] == exposures[k]['GOALTYPE']).all()
    assert (frames['FAPRGRM'][frames['EXPID'] == expid] == exposures[k]['FAPRGRM']).all()
    assert (frames['FAFLAVOR'][frames['EXPID'] == expid] == exposures[k]['FAFLAVOR']).all()
    assert (frames['MINTFRAC'][frames['EXPID'] == expid] == exposures[k]['MINTFRAC']).all()
    assert (frames['GOALTIME'][frames['EXPID'] == expid] == exposures[k]['GOALTIME']).all()

In [ ]:
for expid in frames['EXPID']:
    assert expid in exposures['EXPID']

In [ ]:
program = faflavor2program(exposures['FAFLAVOR'])
for survey in np.unique(exposures['SURVEY']):
    print(f"'{survey}': ", np.unique(program[exposures['SURVEY'] == survey]).tolist(), ',', sep='')

## Test load individual tables

In [ ]:
dsr.log = get_logger(DEBUG, timestamp=True)
postgresql = dsr.setup_db(overwrite=True, hostname='nerscdb03.nersc.gov', username='desidev_admin', schema=specprod, verbose=True)

In [ ]:
loader = [{'filepaths': os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'),
           'tcls': dsr.Exposure,
           'hdu': 'EXPOSURES',
           'expand': None,
           'insert': {'faprgrm': ('program',)},
           'convert': {'program': faflavor2program},
           'index': None,
           'q3c': 'tilera',
           'chunksize': 50000,
           'maxrows': 0
          },
          {'filepaths': os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'),
           'tcls': dsr.Frame,
           'hdu': 'FRAMES',
           'expand': None,
           'insert': None,  # {'night': ('frameid',)},
           'convert': None,  # {'frameid': frameid},
           'index': None,
           'q3c': None,
           'chunksize': 50000,
           'maxrows': 0
          }]


In [ ]:
for l in loader:
    dsr.load_file(**l)